In [8]:
import pandas as pd

# Depending on the metric, the value column contains integer counts or
# percentage fractions in addition to entries marked `N/A`.
TYPES = {
    'app': 'category',
    'policy_area': 'category',
    'metric': 'category',
    'period': 'period[Q]',
    'value': 'string',
}

def loadQ(quarter, year=2022):
    return pd.read_csv(f'data/CSER-{year}-Q{quarter}.csv', dtype=TYPES)

meta = loadQ(3)

# Ideally, diff is empty. In reality, Q4 2020 was recounted
# between Q2 2022 and Q3 2022. It's all rather odd.ß
diff = (
    pd.merge(
        loadQ(2),
        meta,
        how='inner',
        on=['app', 'policy_area', 'metric', 'period'],
        suffixes=('2', '3'),
    )
    .query('value2 != value3')
    .sort_values(['period', 'metric', 'policy_area'])
)

# Identify rows that contain integer values, not percent fractions.
intval = (~ diff['value2'].str.endswith('%')) & (~ diff['value3'].str.endswith('%'))

# Update those rows only with their parsed integer values.

def asint(column):
    def asint2(df):
        return df[column].str.replace(',', '').astype('int')
    return asint2

idiff = diff.loc[intval, :].assign(value2=asint('value2'), value3=asint('value3'))

# For those rows only, compute the percent difference between the values and
# write the result back into the difference frame.
diff.loc[intval, 'diff'] = (
    (idiff['value3'] - idiff['value2']) / idiff['value2'] * 100
).round(1)

# Display the result
print(len(diff), 'rows differ\n')
print(diff.value_counts('period'))

print('\nEntries per quarter', len(meta[meta['period'].astype(str) == '2020Q4']))

dd = diff['diff']
min = dd.min()
min1 = diff[dd > min]['diff'].min()
max = dd.max()

print('\nPercentage difference from v2 to v3:')
print('min diff  ', min)
print('min+1 diff', min1)
print('max diff  ', max)

if False:
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(diff)

113 rows differ

period
2020Q4    77
2022Q2    29
2021Q2     4
2021Q1     3
Freq: Q-DEC, dtype: int64

Entries per quarter 143 


Percentage difference from v2 to v3:
min diff   -50.0
min+1 diff -37.9
max diff   7.7


## Meta: CSAM Pieces per Quarter

In [36]:
CSAM = (
    'Child Nudity & Sexual Exploitation',
    # -- End Q1 2021 -- Start Q2 2021 --
    'Child Endangerment: Sexual Exploitation',
)

csam = (
    meta
    [meta['policy_area'].isin(CSAM) & (meta['metric'] == 'Content Actioned')]
    .drop(columns=['app', 'policy_area', 'metric'])
    .dropna()
    .set_index('period')
    .assign(value=asint('value'))
    .pipe(lambda df: df.groupby(df.index).sum())
)

display(pd.DataFrame({
    'count': csam['value'].map('{:,}'.format)
}))


,count
period,
2018Q3,"9,000,000"
2018Q4,"7,200,000"
2019Q1,"5,800,000"
2019Q2,"7,426,200"
2019Q3,"12,155,800"
2019Q4,"13,986,400"
2020Q1,"9,500,000"
2020Q2,"2,958,200"
2020Q3,"10,770,600"


## Meta: CSAM Pieces per Year

In [38]:
yearly = csam.groupby(csam.index.year).sum()
display(
    yearly['value'].map('{:,}'.format)
)

period
2018    16,200,000
2019    39,368,400
2020    28,187,700
2021    78,012,400
2022    71,000,000
Name: value, dtype: object